In [1]:
import os
import pandas as pd
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from torch import nn
from torchvision import models, transforms
from torchvision.models import VGG19_Weights
import torch
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from PIL import Image
from torchviz import make_dot
from torch.autograd import Variable

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Paths
mat_file_path = "imagelabels.mat"
image_dir = "images"

mat_data = loadmat(mat_file_path)

print("Keys in .mat file:", mat_data.keys())

labels = mat_data['labels'].squeeze()

image_files = sorted(os.listdir(image_dir))

assert len(labels) == len(image_files), "Number of labels and images do not match!"

# create a DataFrame with image names and labels
flowers = pd.DataFrame({
    "Image_Name": image_files,
    "Label": labels
})

Keys in .mat file: dict_keys(['__header__', '__version__', '__globals__', 'labels'])


In [4]:
# Split the dataset into train, validation, and test sets (50%, 25%, 25%)
train_val, test = train_test_split(flowers, test_size=0.25, stratify=flowers['Label'])
train, val = train_test_split(train_val, test_size=0.33, stratify=train_val['Label'])

print(f"Training set: {len(train)}, Validation set: {len(val)}, Test set: {len(test)}")

Training set: 4114, Validation set: 2027, Test set: 2048


In [5]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit pre-trained models
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

class FlowerDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = os.path.join(self.image_dir, row["Image_Name"])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(row["Label"] - 1, dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label


# Create datasets
train_dataset = FlowerDataset(train, image_dir, transform)
val_dataset = FlowerDataset(val, image_dir, transform)
test_dataset = FlowerDataset(test, image_dir, transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss / len(train_loader))

        # Validation phase
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(correct / total)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, "
              f"Val Loss: {val_losses[-1]:.4f}, Val Accuracy: {val_accuracies[-1]:.4f}")

    return train_losses, val_losses, val_accuracies

In [7]:
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Get predictions
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    # Calculate metrics
    accuracy = correct / total
    avg_loss = test_loss / len(test_loader)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy

# VGG19

In [8]:
vgg19 = models.vgg19(weights=VGG19_Weights.DEFAULT)

# Modify VGG19 for flower classification
num_classes = len(flowers['Label'].unique())  # Number of flower classes
vgg19.classifier[6] = torch.nn.Linear(vgg19.classifier[6].in_features, num_classes)
vgg19 = vgg19.to(device)

In [9]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # Batch size 1, 3 color channels, 224x224 resolution
output = vgg19(dummy_input)
graph = make_dot(output, params=dict(vgg19.named_parameters()))
graph.render("vgg19", format="png", cleanup=True)

'vgg19.png'

In [ ]:
# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg19.parameters(), lr=1e-4)

# Train VGG19
train_losses, val_losses, accuracies = train_model(vgg19, criterion, optimizer, train_loader, val_loader, num_epochs=10)

In [ ]:
# Plot Accuracy and Cross-Entropy
plt.figure(figsize=(10, 5))
plt.plot(accuracies, label="Accuracy")
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Cross-Entropy Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [22]:
# Test the trained model
test_loss, test_accuracy = test_model(vgg19, test_loader, criterion, device)
print(f"Final Test Loss: {test_loss:.5f}%")
print(f"Final Test Accuracy: {test_accuracy * 100:.5f}%")

Test Loss: 0.7026, Test Accuracy: 0.8442
Final Test Loss: 0.70261%
Final Test Accuracy: 84.42383%
